## iQuantum Dataset Generator

In [ ]:
import os
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.qasm import Qasm

def analyze_qasm(qasm_path):
    with open(qasm_path, "r") as qasm_file:
        qasm_str = qasm_file.read()
    qc = QuantumCircuit.from_qasm_str(qasm_str)
    n_qubits = qc.num_qubits
    qc_depth = qc.depth()

    single_qubit_gate_names = ["u1", "u2", "u3", "id", "x", "y", "z", "h", "s", "sdg", "t", "tdg", "rx", "ry", "rz", "sx", "measure"]
    multi_qubit_gate_names = ["cx", "ccx"]

    single_qubit_gates = {}
    multi_qubit_gates = {}

    gate_counts = qc.count_ops()

    for gate, count in gate_counts.items():
        if gate in single_qubit_gate_names:
            single_qubit_gates[gate] = count
        elif gate in multi_qubit_gate_names:
            multi_qubit_gates[gate] = count

    topology = set()
    cx_instruction = qc.get_instructions(name="cx")
    for instr, qargs, _ in cx_instruction:
            qubit_pair = tuple([qarg.index for qarg in qargs])  # Sort qubit indices to ensure uniqueness
            topology.add(qubit_pair)
            topology.add(qubit_pair[::-1])  # Add the opposite connection
    topology_sorted = sorted(list(topology))
    topology_str = "; ".join(map(str, topology_sorted))

    return {
        "n_qubits": n_qubits,
        "qc_depth": qc_depth,
        "single_qubit_gates": single_qubit_gates,
        "multi_qubit_gates": multi_qubit_gates,
        "qubit_topology": topology_str,
    }
    
def export_to_csv(data, csv_path):
    df = pd.DataFrame(data)
    df.index.name = "index"
    df.to_csv(csv_path, index=True)

In [ ]:
folder_path = "QASM/MQT-Set2-7-127-AllOpt-IBMMapped"
csv_output_path = "MQT-Set2-7-127-AllOpt-IBMMapped.csv"
qasm_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".qasm")]

analyzed_data = []

for qasm_path in qasm_files:
    # print(f"Processing: {qasm_path}")  # Add this line to print the file being processed
    analysis = analyze_qasm(qasm_path)
    analysis["qasm_file"] = os.path.basename(qasm_path)
    analyzed_data.append(analysis)

export_to_csv(analyzed_data, csv_output_path)

In [6]:
# Post-processing
import csv
import random

input_file = "MQT-Set2-7-127-AllOpt-IBMMapped.csv"
output_file = "MQT-Set2-7-127-AllOpt-IBMMapped-Extra.csv"

# Define the mapping of qasm_file values to mapped_backend values
mapping = {
    "ibm_montreal": "ibm_27q",
    "ibm_washington": "ibm_127q"
}

# Open the input and output files
with open(input_file, "r") as file_in, open(output_file, "w", newline="") as file_out:
    reader = csv.DictReader(file_in)
    fieldnames = reader.fieldnames + ["n_shots", "mapped_backend", "application", "sdk", "opt_level"]  # Add the new fields
    writer = csv.DictWriter(file_out, fieldnames)
    
    writer.writeheader()  # Write the header
    
    for row in reader:
        qasm_file = row["qasm_file"]
        n_shots = 0
        mapped_backend = ""
        application = ""
        sdk = ""
        opt_level = ""
        
        n_shots = random.randint(1024, 4096)  # Generate a random number between 1024 and 4096 (inclusive)

        # Check if the qasm_file contains the specific keywords and update the corresponding fields accordingly
        if "ibm_montreal" in qasm_file:
            mapped_backend = mapping["ibm_montreal"]
        elif "ibm_washington" in qasm_file:
            mapped_backend = mapping["ibm_washington"]
        
        if "_" in qasm_file:
            application = qasm_file.split("_")[0]
        
        if "qiskit" in qasm_file:
            sdk = "qiskit"
        
        if "opt" in qasm_file:
            opt_level = qasm_file.split("opt")[-1].split("_")[0]
        row["n_shots"] = n_shots  # Add the n_shots value to the row
        row["mapped_backend"] = mapped_backend  # Add the mapped_backend value to the row
        row["application"] = application  # Add the application value to the row
        row["sdk"] = sdk  # Add the sdk value to the row
        row["opt_level"] = opt_level  # Add the opt_level value to the row
        
        writer.writerow(row)  # Write the updated row to the output file
